In [ ]:
import numpy as np
import pandas as pd

# import yfinance as yf
from tvDatafeed import TvDatafeed, Interval

import warnings
warnings.filterwarnings("ignore")

from datetime import datetime
import time

In [ ]:
tv = TvDatafeed()

# portfolio of stocks
# FPMARKETS:US100
# CAPITALCOM:US500
# FOREXCOM:US30
oex = ['US100', 'US500', 'US30']

num_stocks = len(oex)

start_date = datetime(2015, 1, 1)
today = datetime.now()

days_difference = (today - start_date).days

n_bars = min(days_difference + 100, 5000)  # max 5000 API limit

# stocks and their exchanges
stock_exchanges = {
    'US100': 'FPMARKETS',
    'US500': 'CAPITALCOM',
    'US30': 'FOREXCOM'
}

# function to download data with retries
def download_with_retry(symbol, exchange, retries=3, delay=2):
    for attempt in range(retries):
        try:
            print(f"Downloading {symbol} from {exchange} (attempt {attempt+1})")
            data = tv.get_hist(
                symbol=symbol, 
                exchange=exchange, 
                interval=Interval.in_daily,
                n_bars=1000  # circa 4 anni di dati
            )
            if data is not None and not data.empty:
                print(f"✅ {symbol}: {len(data)} rows downloaded")
                return data
        except Exception as e:
            print(f"❌ Error downloading {symbol}: {e}")
        
        if attempt < retries - 1:
            time.sleep(delay)
    
    print(f"❌ Failed to download {symbol} after {retries} attempts")
    return None

# Download data for all stocks
print("🚀 Starting data download...")
data_dict = {}
failed_downloads = []

for stock, exchange in stock_exchanges.items():
    data = download_with_retry(stock, exchange)
    if data is not None:
        data_dict[stock] = data['close']
    else:
        failed_downloads.append(stock)

if failed_downloads:
    print(f"⚠️ Failed to download: {failed_downloads}")

# Combine into a single DataFrame
if data_dict:
    df = pd.DataFrame(data_dict)
    print(f"📊 Combined DataFrame shape: {df.shape}")
    print(f"📅 Date range: {df.index.min()} to {df.index.max()}")
    print("\nFirst 5 rows:")
    print(df.head())
else:
    print("❌ No data downloaded successfully")